In [1]:
import pandas as pd
import json
import os

In [2]:
stations = pd.read_csv('data/postaje.txt', sep=',')
stations

,NAME,LAT,LON
0,LIDL BEŽIGRAD,46.063797,14.506854
1,ŠMARTINSKI PARK,46.065206,14.529911
2,SAVSKO NASELJE 1-ŠMARTINSKA CESTA,46.062475,14.524321
3,ČRNUČE,46.102446,14.530213
4,VILHARJEVA CESTA,46.060050,14.513020
...,...,...,...
79,AMBROŽEV TRG,46.049877,14.516308
80,VOJKOVA - GASILSKA BRIGADA,46.068727,14.516858
81,RAKOVNIK,46.036284,14.522948
82,PREGLOV TRG,46.054554,14.559080


In [3]:
# create a pd.DataFrame from the stations data with a time column
# and a column for each station
df = pd.DataFrame(columns=['time'] + list(stations['NAME']))
df
            

,time,LIDL BEŽIGRAD,ŠMARTINSKI PARK,SAVSKO NASELJE 1-ŠMARTINSKA CESTA,ČRNUČE,VILHARJEVA CESTA,MASARYKOVA DDC,POGAČARJEV TRG-TRŽNICA,CANKARJEVA UL.-NAMA,ANTONOV TRG,...,BTC CITY ATLANTIS,IKEA,MIKLOŠIČEV PARK,BARJANSKA C.-CENTER STAREJŠIH TRNOVO,LEK - VEROVŠKOVA,AMBROŽEV TRG,VOJKOVA - GASILSKA BRIGADA,RAKOVNIK,PREGLOV TRG,PLEČNIKOV STADION


In [4]:
prev_row = None


for filename in os.listdir('data'):
    if filename.endswith(".json") == False:
        continue

    with open('data/' + filename, encoding="utf-8") as file:
        TIME = filename.split('.')[0]
        
        
        try:
            data = json.load(file)
        except:
            prev_row['time'] = TIME
            df = pd.concat([df, prev_row], ignore_index=True)

        stations_values_dict = {}
        for station in stations['NAME']:
            stations_values_dict[station] = None

        for station in data:
            NAME = station['name']
            AVAILABLE_BIKES = station['totalStands']['availabilities']['bikes']

            if NAME not in df.columns:
                continue

            stations_values_dict[NAME] = AVAILABLE_BIKES

        for station in stations['NAME']:
            if station not in stations_values_dict:
                stations_values_dict[station] = None

        stations_values_dict['time'] = TIME
        c_df = pd.DataFrame(stations_values_dict, index=[0])
        prev_row = c_df

        df = pd.concat([df, c_df], ignore_index=True)

df.head()
                

,time,LIDL BEŽIGRAD,ŠMARTINSKI PARK,SAVSKO NASELJE 1-ŠMARTINSKA CESTA,ČRNUČE,VILHARJEVA CESTA,MASARYKOVA DDC,POGAČARJEV TRG-TRŽNICA,CANKARJEVA UL.-NAMA,ANTONOV TRG,...,BTC CITY ATLANTIS,IKEA,MIKLOŠIČEV PARK,BARJANSKA C.-CENTER STAREJŠIH TRNOVO,LEK - VEROVŠKOVA,AMBROŽEV TRG,VOJKOVA - GASILSKA BRIGADA,RAKOVNIK,PREGLOV TRG,PLEČNIKOV STADION
0,2022-10-12-18-05,9,4,10,6,8,4,2,13,19,...,16,11,1,7,0,4,1,9,15,0
1,2022-10-12-18-10,10,4,10,6,8,3,0,3,19,...,16,11,0,7,0,2,1,11,15,0
2,2022-10-12-18-15,10,3,10,6,9,3,1,6,20,...,16,12,2,7,0,4,1,12,15,0
3,2022-10-12-18-20,7,3,9,7,8,2,0,3,20,...,16,11,6,7,1,3,0,13,15,0
4,2022-10-12-18-25,5,2,9,7,8,6,1,3,19,...,16,13,6,6,0,5,0,13,15,3


In [5]:
# DUMP TO bike_data.csv
df.to_csv('bike_data.csv', index=False, encoding='utf-8', sep=';')